In [30]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(111)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [36]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [39]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [42]:
# dataset Loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [45]:
# nn Layers
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [48]:
# Initialization
torch.nn.init.normal_(linear1.weight)
torch.nn.init.normal_(linear2.weight)
torch.nn.init.normal_(linear3.weight)

Parameter containing:
tensor([[ 3.5803e-01, -1.5184e-01,  1.1797e-01,  ..., -1.0853e+00,
         -6.2846e-01, -4.2736e-01],
        [-5.5581e-02, -1.1031e+00, -1.3390e-01,  ..., -2.0440e+00,
         -1.5062e+00, -1.7622e+00],
        [-8.0880e-01,  7.1884e-01, -1.9147e-01,  ..., -9.6052e-01,
          2.0410e+00, -7.1375e-01],
        ...,
        [ 1.5810e-03,  1.5663e-01, -6.2225e-01,  ..., -1.8665e+00,
         -3.1233e-01,  2.0360e+00],
        [ 1.0034e+00,  3.1570e-01, -2.2853e-01,  ..., -1.3537e+00,
          4.3089e-01, -1.2895e+00],
        [-8.4354e-01, -1.0107e+00, -8.3191e-01,  ...,  1.7194e+00,
          9.3720e-02, -8.5864e-01]], requires_grad=True)

In [51]:
# model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [54]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [55]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # Label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
    
print('Learning finished')

Epoch: 0001 cost = 0.926895380
Epoch: 0002 cost = 0.743556559
Epoch: 0003 cost = 0.627000630
Epoch: 0004 cost = 0.535983205
Epoch: 0005 cost = 0.497211397
Epoch: 0006 cost = 0.503066361
Epoch: 0007 cost = 0.408927530
Epoch: 0008 cost = 0.357866585
Epoch: 0009 cost = 0.330913931
Epoch: 0010 cost = 0.383246154
Epoch: 0011 cost = 0.355180025
Epoch: 0012 cost = 0.318381846
Epoch: 0013 cost = 0.353953809
Epoch: 0014 cost = 0.296688497
Epoch: 0015 cost = 0.310768902
Learning finished


In [58]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
    
    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)
    
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

/Users/emptyhead/opt/anaconda3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9627000093460083
Label:  9
Prediction:  9
